## 1. 초기 세팅

In [ ]:
import argparse
import bottleneck as bn
from copy import deepcopy
import time
import os
import multiprocessing

import numpy as np
from scipy import sparse
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import ndcg_score, recall_score

In [ ]:
## 각종 파라미터 세팅
parser = argparse.ArgumentParser(description='PyTorch RecVAE')


parser.add_argument('--data', type=str, default='../../data/train/',
                    help='Movielens dataset location')

parser.add_argument('--lr', type=float, default=5e-4,
                    help='initial learning rate')
parser.add_argument('--gamma', type=float, default=0.005)

parser.add_argument('--batch_size', type=int, default=100,
                    help='batch size')
parser.add_argument('--epochs', type=int, default=50,
                    help='upper epoch limit')

parser.add_argument('--n-enc_epochs', type=int, default=3)

parser.add_argument('--n-dec_epochs', type=int, default=1)

parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--cuda', action='store_true',
                    help='use CUDA')

parser.add_argument('--save_dir', type=str, default='./',
                    help='path to save the final model')
args = parser.parse_args([])

# Set the random seed manually for reproductibility.
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)

#만약 GPU가 사용가능한 환경이라면 GPU를 사용
if torch.cuda.is_available():
    args.cuda = True

device = torch.device("cuda" if args.cuda else "cpu")
device

## 2. 데이터 전처리

In [ ]:
#훈련된 모델을 이용해 검증할 데이터를 분리하는 함수입니다.
#100개의 액션이 있다면, 그중에 test_prop 비율 만큼을 비워두고, 그것을 모델이 예측할 수 있는지를
#확인하기 위함입니다.
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('user')
    tr_list, te_list = list(), list()
    
    for _, group in data_grouped_by_user:
        n_items_u = len(group)
        
        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        
        else:
            print(f"n_items_user is lower than 5, it is {n_items_u}")
            tr_list.append(group)
    
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te

def numerize(df, user2id, item2id):
    user = df['user'].apply(lambda x: user2id[x])
    item = df['item'].apply(lambda x: item2id[x])
    return pd.DataFrame(data={'user': user, 'item': item}, columns=['user', 'item'])

In [ ]:
# Load Data
DATA_DIR = args.data
raw_data = pd.read_csv(os.path.join(args.data, 'train_ratings.csv'), header=0)
print("원본 데이터\n", raw_data)

user_review_count = raw_data[["user"]].groupby("user", as_index=False).size()
item_review_count = raw_data[["item"]].groupby("item", as_index=False).size()
print("유저별 리뷰수\n", user_review_count)
print("아이템별 리뷰수\n",item_review_count)

In [ ]:
# Shuffle User Indices
unique_user_id = raw_data["user"].unique()
unique_item_id = raw_data["item"].unique()

item2id = dict((item_id, i) for (i, item_id) in enumerate(unique_item_id))
user2id = dict((user_id, i) for (i, user_id) in enumerate(unique_user_id))


# print("(BEFORE) unique_user_id:", unique_user_id)
idx_perm = np.random.permutation(unique_user_id.size)
unique_user_id = unique_user_id[idx_perm]
# print("(AFTER) unique_user_id:",unique_user_id)

num_users = unique_user_id.size #31360
num_items = unique_item_id.size #6807
print(f"전체 유저 수, 전체 영화 수: {num_users}, {num_items}")

# Split Train/Validation/Test User Indices
valid_users = unique_user_id[-int(num_users * 0.2):]
train_users = unique_user_id[:int(num_users * 0.8)]

#주의: 데이터의 수가 아닌 사용자의 수입니다!
print("훈련 데이터에 사용될 사용자 수:", len(train_users))
print("검증 데이터에 사용될 사용자 수:", len(valid_users))

In [ ]:
##훈련 데이터에 해당하는 아이템들
#Train에는 전체 데이터를 사용합니다.
train_data = raw_data.loc[raw_data['user'].isin(train_users)]
train_data = numerize(train_data, user2id, item2id)

#Validation으로 사용될 tr 데이터와 정답을 확인하기 위한 te 데이터로 분리되었습니다.
valid_data = raw_data.loc[raw_data['user'].isin(valid_users)]
valid_data_tr, valid_data_te = split_train_test_proportion(valid_data)
valid_data_tr = numerize(valid_data_tr, user2id, item2id)
valid_data_te = numerize(valid_data_te, user2id, item2id)

total_data = numerize(raw_data, user2id, item2id)

In [ ]:
train_data, valid_data_tr, valid_data_te

## 3. Dataset 설정

In [ ]:
class VAEDataset(Dataset):
    def __init__(self, X, Y=None, num_users=31360, num_items=6807):
        self.num_users = num_users
        self.num_items = num_items
        self.X = self._data_to_tensor(X)
        if Y is not None:
            self.Y = self._data_to_tensor(Y)
        else:
            self.Y = self.X
        
    
    def _data_to_tensor(self, data, user='user', item='item'):
        matrix = np.zeros((self.num_users, self.num_items))
        matrix[data[user].values, data[item].values] = 1.0
        matrix = matrix[np.any(matrix, axis=1)]
        return torch.FloatTensor(matrix)
    
    def __len__(self):
        return self.X.size()[0]
    
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

## 4. 모델정의



In [ ]:
# RecVAE

def swish(x):
    return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
    return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())


class CompositePrior(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
        super(CompositePrior, self).__init__()
        
        self.mixture_weights = mixture_weights
        
        self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.mu_prior.data.fill_(0)
        
        self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_prior.data.fill_(0)
        
        self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_uniform_prior.data.fill_(10)
        
        self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
        self.encoder_old.requires_grad_(False)
        
    def forward(self, x, z):
        post_mu, post_logvar = self.encoder_old(x, 0)
        
        stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
        post_prior = log_norm_pdf(z, post_mu, post_logvar)
        unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)
        
        gaussians = [stnd_prior, post_prior, unif_prior]
        gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]
        
        density_per_gaussian = torch.stack(gaussians, dim=-1)
                
        return torch.logsumexp(density_per_gaussian, dim=-1)

    
class Encoder(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
        super(Encoder, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, dropout_rate):
        norm = x.pow(2).sum(dim=-1).sqrt()
        x = x / norm[:, None]
        # x[x != x] = 0
    
        x = F.dropout(x, p=dropout_rate, training=self.training)
        
        h1 = self.ln1(swish(self.fc1(x)))
        h2 = self.ln2(swish(self.fc2(h1) + h1))
        h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
        h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
        h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
        return self.fc_mu(h5), self.fc_logvar(h5)
    

class RecVAE(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim):
        super(RecVAE, self).__init__()

        self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
        self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, user_ratings, beta=None, gamma=1, dropout_rate=0.5, calculate_loss=True):
        mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)    
        z = self.reparameterize(mu, logvar)
        x_pred = self.decoder(z)       
        
        if gamma:
            norm = user_ratings.sum(dim=-1)
            kl_weight = gamma * norm
        elif beta:
            kl_weight = beta

        mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
        kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
        negative_elbo = -(mll - kld)
        
        return negative_elbo, x_pred


    def update_prior(self):
        self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))


## 5. Metric (recall@k)

In [ ]:
def recall_at_k_batch(X_pred, heldout_batch, k=10):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0)
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    return tmp / np.minimum(k, X_true_binary.sum(axis=1))

## 6. Dataloader

In [ ]:
###############################################################################
# Load data
###############################################################################

train_dataloader = DataLoader(VAEDataset(train_data),
                              batch_size=args.batch_size,
                              shuffle=True,
                              num_workers=multiprocessing.cpu_count() // 2)

valid_dataloader = DataLoader(VAEDataset(valid_data_tr, valid_data_te),
                              batch_size=len(valid_users), 
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count() // 2)

total_dataloader = DataLoader(VAEDataset(total_data),
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=multiprocessing.cpu_count() // 2)

## 7. Build Model & Run Train

In [ ]:
# ###############################################################################
# # Build the model
# ###############################################################################
args.epochs = 200
args.lr = 5e-4
args.gamma = 0.005
args.batch_size = 500

latent_dim, hidden_dim, input_dim = [200, 600, num_items]
model = RecVAE(hidden_dim, latent_dim, input_dim).to(device)

enc_optimizer = optim.Adam(model.encoder.parameters(), lr=args.lr)
dec_optimizer = optim.Adam(model.decoder.parameters(), lr=args.lr)

model_params = {'hidden_dims': (latent_dim, hidden_dim, input_dim),
                "optimizer": enc_optimizer,
                "enc_epoch / dec_epoch": 3,
                "gamma": args.gamma
               }

args.save_dir = f'./output/epoch{args.epochs}_adam{args.lr}_gamma{args.gamma}_batch{args.batch_size}_hidden{(latent_dim, hidden_dim, input_dim)}'
os.makedirs(args.save_dir, exist_ok=True)

model_params, device, args.save_dir

In [ ]:
###############################################################################
# Training code
###############################################################################
best_r10 = -np.inf
best_epoch = 0
train_loss_list = list()
valid_loss_list = list()
valid_r10_list = list()
valid_r20_list = list()

for epoch in range(1, args.epochs + 1):
    epoch_start_time = time.time()
    
    model.train()
    train_loss = 0.0
    
    # train
    for x, y in train_dataloader:
        x, y = x.to(device), y.to(device)
        
        # encoder
        for i in range(args.n_enc_epochs):
            enc_optimizer.zero_grad()
            loss, _ = model(x, gamma=args.gamma, dropout_rate=0.5)
            loss.backward()
            enc_optimizer.step()
        
        model.update_prior()
        
        #decoder
        for i in range(args.n_dec_epochs):
            dec_optimizer.zero_grad()
            loss, _ = model(x, gamma=args.gamma, dropout_rate=0.0)
            loss.backward()
            dec_optimizer.step()
        
        train_loss += loss.item()
    train_loss_list.append(train_loss / len(train_dataloader))
    
    model.eval()
    for x, y in valid_dataloader:
        x, y = x.to(device), y.to(device)
        
        val_loss, pred = model(x, calculate_loss=False)
        
        # Exclude examples from training set
        pred = pred.detach().cpu().numpy()
        pred[x.detach().cpu().numpy().nonzero()] = -np.inf

        r10 = recall_at_k_batch(pred, y.detach().cpu().numpy(), 10).mean()
        r20 = recall_at_k_batch(pred, y.detach().cpu().numpy(), 20).mean()
    
    valid_loss_list.append(val_loss.item())
    valid_r10_list.append(r10)
    valid_r20_list.append(r20)
        
    print(f"| end of epoch {epoch} | time: {time.time() - epoch_start_time:4.2f}s "
          f"| valid loss {val_loss:4.2f} "
          f"| r10 {r10:6.4f} "
          f"| r20 {r20:6.4f}")


    # Save the model if the n100 is the best we've seen so far.
    if r10 > best_r10:
        with open(os.path.join(args.save_dir, "model.pt"), 'wb') as f:
            torch.save(model, f)
        best_r10 = r10
        best_epoch = epoch

## 8. Loss 변화 확인

In [ ]:
# plt.plot(range(1, args.epochs + 1), train_loss_list, label="train_loss")
plt.plot(range(1, len(valid_loss_list) + 1), valid_loss_list, label="valid_loss")
plt.legend()
plt.savefig(os.path.join(args.save_dir, "loss.png"))

In [ ]:
plt.plot(range(1, len(valid_r10_list) + 1), valid_r10_list, label="r10")
plt.plot(range(1, len(valid_r20_list) + 1), valid_r20_list, label="r20")
plt.legend()
plt.savefig(os.path.join(args.save_dir, f"recall_best{best_epoch}.png"))

## 9. output file 생성

In [ ]:
# total_data
best_model = torch.load(os.path.join(args.save_dir, 'model.pt')).to(device)

In [ ]:
item_output = list()
best_model.eval()

for x, _ in tqdm(total_dataloader):
    x = x.to(device)
    
    _, pred_out = best_model(x, calculate_loss=False)
    pred_out = pred_out.detach().cpu().numpy()
    pred_out[x.detach().cpu().numpy().nonzero()] = -np.inf
    
    idxs = bn.argpartition(-pred_out, 10, axis=1)
    item_output.extend(idxs[: ,:10].reshape(-1, ))
len(item_output)

In [ ]:
sub_items = np.array(sorted(item2id.items(), key=lambda x: x[1]))[item_output, 0]
sub_items[:10]

In [ ]:
movies_df = pd.read_csv("../../movies_info.csv")
movies_df.loc[movies_df.item.isin(sub_items[:10])]

In [ ]:
submission = pd.read_csv("../../data/eval/sample_submission.csv")
submission["item"] = sub_items
submission.to_csv(os.path.join(args.save_dir,
                               f"./RecVAE_epoch{args.epochs}_adam{args.lr}_hidden{model_params['hidden_dims']}.csv"),
                  index=False)

## Opt. output.csv with score

In [ ]:
# with score
best_model = torch.load(os.path.join('./output/epoch500_adam0.0005_batch500_hidden(200, 600, 6807)/', 'with_total_dataset_model.pt'))
best_model = best_model.to(device)

In [ ]:
inference_dataloader = DataLoader(VAEDataset(total_data),
                                  batch_size=3000,
                                  shuffle=False,
                                  num_workers=multiprocessing.cpu_count() // 2)

In [ ]:
# top 20
from sklearn.preprocessing import MinMaxScaler

K = 50
scaler = MinMaxScaler()
item_output = list()
best_model.eval()

for x, _ in tqdm(inference_dataloader):
    x = x.to(device)
    
    _, pred_out = best_model(x, calculate_loss=False)
    pred_out = pred_out.detach().cpu().numpy()
    pred_out[x.detach().cpu().numpy().nonzero()] = -np.inf

    for pred in pred_out:
        pred_item_ids = np.where(pred != -np.inf)[0]
        pred = scaler.fit_transform(pred[pred_item_ids].reshape(-1, 1)).reshape(-1, )

        idx_score = np.vstack((pred_item_ids, pred)).T
        idx_score = idx_score[idx_score[:, 1].argsort()[::-1]][:K]
        
        item_output.extend(idx_score.astype(float))

len(item_output)

In [ ]:
%time
sub_users = np.sort(unique_user_id).repeat(K)
item_output = np.array(item_output)
sub_items = np.array(sorted(item2id.items(), key=lambda x: x[1]))[item_output[:, 0].astype(int), 0]
sub_scores = item_output[:, 1]

In [ ]:
movies_df = pd.read_csv("../../movies_info.csv")
sub_items[:K]
movies_df.loc[movies_df.item.isin(sub_items[:K])]

In [ ]:
%time
submission = pd.DataFrame(0, index=np.arange(sub_users.shape[0]), columns=["user", "item", "score"])

submission.user = sub_users
submission.item = sub_items
submission.score = sub_scores

In [ ]:
%time
submission.to_csv(os.path.join('./output/epoch500_adam0.0005_batch500_hidden(200, 600, 6807)/',
                               f"./RecVAE_score{K}_epoch500_adam0.0005_hidden(200, 600, 6807).csv"),
                  index=False)